# Tutorial

[![Open in Colab][colab_badge]][colab_link] [![Binder][binder_badge]][binder_link]

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_link]: https://colab.research.google.com/github/dustalov/evalica/blob/master/docs/tutorial.ipynb
[binder_badge]: https://mybinder.org/badge_logo.svg
[binder_link]: https://mybinder.org/v2/gh/dustalov/evalica/HEAD?labpath=docs/tutorial.ipynb

In [ ]:
import evalica
import pandas as pd
import plotly.express as px
from evalica import Winner, alpha_bootstrap, bootstrap, bradley_terry

%config InlineBackend.figure_formats = ['svg']

In [ ]:
evalica.__version__

## Pairwise Comparisons

In [ ]:
df_food = pd.read_csv(
    "https://raw.githubusercontent.com/dustalov/evalica/0893fd0f1e8107b2d62fd6c5816b55b417c1a050/food.csv",
    dtype=str,
)

df_food["winner"] = df_food["winner"].map(
    {
        "left": Winner.X,
        "right": Winner.Y,
        "tie": Winner.Draw,
    },
)

df_food.head(5)

In [ ]:
df_food["left_id"], df_food["right_id"], index = evalica.indexing(df_food["left"], df_food["right"])

In [ ]:
matrices = evalica.matrices(df_food["left_id"], df_food["right_id"], df_food["winner"], index)

In [ ]:
pd.DataFrame(matrices.win_matrix, index=index, columns=index)  # win matrix

In [ ]:
pd.DataFrame(matrices.tie_matrix, index=index, columns=index)  # tie matrix

In [ ]:
count_result = evalica.counting(df_food["left"], df_food["right"], df_food["winner"])
count_result.scores.to_frame()

In [ ]:
avr_result = evalica.average_win_rate(df_food["left"], df_food["right"], df_food["winner"])
avr_result.scores.to_frame()

In [ ]:
bt_result = evalica.bradley_terry(df_food["left"], df_food["right"], df_food["winner"])
bt_result.scores.to_frame()

In [ ]:
newman_result = evalica.newman(df_food["left"], df_food["right"], df_food["winner"])
newman_result.scores.to_frame()

In [ ]:
eigen_result = evalica.eigen(df_food["left"], df_food["right"], df_food["winner"])
eigen_result.scores.to_frame()

In [ ]:
elo_result = evalica.elo(df_food["left"], df_food["right"], df_food["winner"])
elo_result.scores.to_frame()

In [ ]:
df_bt_pairwise = evalica.pairwise_frame(bt_result.scores)

df_bt_pairwise

In [ ]:
fig = px.imshow(df_bt_pairwise, color_continuous_scale="RdBu", text_auto=".2f")
fig.update_layout(xaxis_title="Loser", yaxis_title="Winner", xaxis_side="top")
fig.update_traces(hovertemplate="Winner: %{y}<br>Loser: %{x}<br>Fraction of Wins: %{z}")
fig.show()

In [ ]:
bootstrap_result = bootstrap(
    bradley_terry,
    df_food["left"],
    df_food["right"],
    df_food["winner"],
    n_resamples=10,
    random_state=42,
)

df_melted = bootstrap_result.distribution.melt(var_name="Item", value_name="Score")

df_melted.head(5)

In [ ]:
fig = px.box(df_melted, x="Score", y="Item", color="Item", title="Bradley–Terry Bootstrap Scores")
fig.update_traces(hovertemplate="<b>%{y}</b><br>Score: %{x:.3f}<extra></extra>")
fig.show()

## Inter-Rater Reliability

In [ ]:
df_codings = pd.read_csv(
    "https://raw.githubusercontent.com/dustalov/evalica/d356c3988fdf9c1db249767413a7a8a1f49d64c0/codings.csv",
    header=None,
    dtype=str,
)

df_codings

In [ ]:
distances = ["nominal", "ordinal", "interval", "ratio"]
alpha_values = {dist: evalica.alpha(df_codings, distance=dist).alpha for dist in distances}  # type: ignore[arg-type]

pd.Series(alpha_values, name="alpha").to_frame()

### Confidence Intervals

Evalica can also compute confidence intervals for Krippendorff's alpha using bootstrapping.

In [ ]:
alpha_bootstrap_result = alpha_bootstrap(
    df_codings,
    distance="nominal",
    n_resamples=1000,
    confidence_level=0.95,
    random_state=42,
)

fig = px.histogram(
    alpha_bootstrap_result.distribution,
    nbins=50,
    title="Krippendorff's Alpha Bootstrap Distribution",
    labels={"value": "Alpha", "count": "Frequency"},
)

fig.add_vline(x=alpha_bootstrap_result.alpha, line_dash="dash", line_color="red", annotation_text="Point Estimate")
fig.add_vline(x=alpha_bootstrap_result.low, line_dash="dot", line_color="blue", annotation_text="Lower Bound")
fig.add_vline(x=alpha_bootstrap_result.high, line_dash="dot", line_color="blue", annotation_text="Upper Bound")
fig.show()